## Prepare OSM data and convert it into routable graph

In [1]:
%load_ext autoreload
%autoreload 2
%cd C:\emobility
%matplotlib inline

C:\emobility


In [2]:
# Load libs
import pandas as pd
import os
import geopandas as gpd
import dataworkers as dw
from shapely.geometry import box
import requests as req
from pprint import pprint
from lib import routing

### 1. Prepare .poly file for cropping OSM data

In [3]:
# Compute the boundary of VG car agents' activities
df_act = pd.read_csv('dbs/agents/1_activity_plans.csv')
df_act.head()

,person,act_id,deso,POINT_X,POINT_Y,act_purpose,mode,dep_time,trav_time,trav_time_min,speed,distance,act_start,act_time,act_end,score
0,10000013,0,1465A0030,378925.8910,6.381022e+06,home,NaN,0.000000,00:00:00,0.000000,NaN,0.000000,0,252.850000,4.214167,121.286636
1,10000013,1,1490C1390,378291.1037,6.400375e+06,other,car,4.214167,00:35:55,35.916667,41.474452,24.827068,289,44.250000,5.550278,121.286636
2,10000013,2,1490C1380,377640.6586,6.400555e+06,work,car,5.550278,00:03:39,3.650000,17.012101,1.034903,337,573.200000,15.164444,121.286636
3,10000013,3,1465A0030,378925.8910,6.381022e+06,home,car,15.164444,01:08:43,68.716667,28.837628,33.027095,979,461.400000,23.999722,121.286636
4,10000025,0,1465A0030,378555.2118,6.378969e+06,home,NaN,0.000000,00:00:00,0.000000,NaN,0.000000,0,268.566667,4.476111,107.917837


In [5]:
gdf_act = dw.df2gdf_point(df_act, 'POINT_X', 'POINT_Y', crs=3006, drop=True).to_crs(4326)
geom = box(*gdf_act.total_bounds)
gdf_box = gpd.GeoDataFrame([1], geometry=[geom], crs=4326)
gdf_box = gdf_box.rename(columns={0: 'box'})
gdf_box

,box,geometry
0,1,"POLYGON ((18.44017 55.36437, 18.44017 61.84580..."


In [16]:
dw.gdf2poly(gdf=gdf_box, targetfile='dbs/vg_car_agents_act_bd.poly')

### 2. Use osmosis to crop the data
osmosis \--read-pbf file="sweden-latest.osm.pbf" \--bounding-polygon file="vg_car_agents_act_bd.poly" \--write-pbf file="sweden_vg_car_users.osm.pbf"

## 3. Try OTP with elevation as the routing engine

In [27]:
# 4 Build routable OTP graph
region = 'vg'
otp_file = 'dbs/otp-2.2.0-shaded.jar'
otp_folder =  f'dbs/{region}'
memory_gb = 63 # assigned memory for java in GB
if os.path.exists(otp_folder + '/Graph.obj') is not True:
    print(f"{region}: build routable OTP graph.")
    routing.otp_build(otp_file=otp_file, otp_folder=otp_folder, memory_gb=memory_gb)
else:
    print(f"Graph.obj for {region} exists. Graph building skipped.")

vg: build routable OTP graph.


Start playing with OTP API from here below.

In [3]:
country = 'sweden'
region = 'vg'
# Start the server
print('Start the OTP server...')
otp_file = 'dbs/otp-2.2.0-shaded.jar'
otp_folder = f'dbs/{region}'
memory_gb = 40  # assigned memory for java in GB
routing.otp_server_starter(otp_file=otp_file, otp_folder=otp_folder, memory_gb=memory_gb)

Start the OTP server...


In [4]:
fromPlace = (59.40223829, 18.07072809)
toPlace = (59.33297, 18.04428)
maxWalkDistance = 300
mode = "BICYCLE,WALK"


request = routing.req_define(fromPlace=fromPlace,
                     toPlace=toPlace,
                     maxWalkDistance=maxWalkDistance,
                     time="12:00am",
                     date="3-24-2023",
                     mode=mode)

In [5]:
request

'http://localhost:8080/otp/routers/default/plan?fromPlace=59.40223829, 18.07072809&toPlace=59.33297, 18.04428&numItineraries=1&clampInitialWait=0&time=12:00am&date=3-24-2023&mode=BICYCLE,WALK&maxWalkDistance=300&bikeSpeed=25&arriveBy=false'

In [6]:
resp = req.get(request)
output = resp.json()

In [36]:
plan = output["plan"]["itineraries"][0]
pprint(plan)

{'arrivedAtDestinationWithRentedBicycle': False,
 'duration': 817,
 'elevationGained': 292.59,
 'elevationLost': 287.83,
 'endTime': 1679613217000,
 'fare': {'details': {}, 'fare': {}},
 'generalizedCost': 1227,
 'legs': [{'agencyTimeZoneOffset': 3600000,
           'alerts': [{'alertHeaderText': 'Unpaved surface'}],
           'arrivalDelay': 0,
           'departureDelay': 0,
           'distance': 10677.68,
           'duration': 817.0,
           'endTime': 1679613217000,
           'from': {'departure': 1679612400000,
                    'lat': 59.4022383,
                    'lon': 18.0707281,
                    'name': 'Origin',
                    'vertexType': 'NORMAL'},
           'generalizedCost': 1227,
           'interlineWithPreviousLeg': False,
           'legElevation': '0,22.4,2,22.4,2,22.4,12,22.3,22,22.3,32,22.2,42,22.2,52,22.2,62,22.1,72,21.9,82,21.8,92,21.6,102,21.3,112,21.1,122,21.0,132,21.2,142,21.4,152,21.7,162,22.0,176,22.4,186,21.9,196,21.4,206,21.2,216,21.1